To save the artifacts and keep all the information in an orderly way, we need to execute the following code in the terminal: </n>

```
mlflow ui --backend-store-uri sqlite:///mlflow.db
```

You can add a relative path if you want. Remember that ```sqlite:///``` is needed to create the database

In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


import sys
sys.path.append("..")

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///../experiments/mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/mnt/d/projects/mlops_example/nb/mlruns/1', creation_time=1718519409655, experiment_id='1', last_update_time=1718519409655, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

Page to download the data --> [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

# Short Preprocessing

In [2]:
# Trip_type
# 1= Street-hail
# 2= Dispatch

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df.loc[:, 'lpep_dropoff_datetime'] = pd.to_datetime(df.lpep_dropoff_datetime)
    df.loc[:, 'lpep_pickup_datetime'] = pd.to_datetime(df.lpep_pickup_datetime)

    # Duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 1) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet").sample(frac=0.8, random_state=42)

In [5]:
print('dataset to train', len(df_train))
print('dataset to val', len(df_val))

dataset to train 73908
dataset to val 49537


In [6]:
# We can attempt some feature engineering to improve the MAE

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
#categorical = ['PULocationID', 'DOLocationID'] initial approach
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts) # Matrix used to train the model


val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# MLflow

![Logging ways using MLflow](../images/MLFlowLogging.png)

![MLflowformats](../images/MLflowFormats.png)

## Lasso - ML Flow basic register - MLflow log model as an artifact ```.log_artifact```

In [37]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "David")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "../data/green_tripdata_2021-02.parquet).sample(frac=0.8, random_state=42)")


    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)

    print('MSE', mse)
    print('MAE', mae)

    mlflow.log_metric("mae", mae)
    
    mlflow.log_artifact(local_path='../models/lin_reg.bin', artifact_path="models_pickle")

MSE 147.28779876137895
MAE 9.330878725855305


## XGboost - ML Flow hyperparameters optimization

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [30]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")], # xgboost and hyperopt try to minimize the error in the validation set
            early_stopping_rounds=3 # If we have 50 or more iteration without improvement in the validation set, so it will stop
        )

        y_pred = booster.predict(valid)
        mae = mean_absolute_error(y_val, y_pred)
        mlflow.log_metric("mae", mae)

    return {'loss': mae, 'status': STATUS_OK}

In [33]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) -> [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo = tpe.suggest, # algorithm choosen to optimize
    max_evals=7, # number of iterations
    trials=Trials()
)

[0]	validation-rmse:11.73691                         
[1]	validation-rmse:11.30256                         
[2]	validation-rmse:10.90288                         
[3]	validation-rmse:10.53423                         
[4]	validation-rmse:10.19262                         
[5]	validation-rmse:9.87947                          
[6]	validation-rmse:9.59004                          
[7]	validation-rmse:9.32497                          
[8]	validation-rmse:9.08069                          
[9]	validation-rmse:8.86032                          
[10]	validation-rmse:8.65176                         
[11]	validation-rmse:8.47085                         
[12]	validation-rmse:8.29496                         
[13]	validation-rmse:8.14359                         
[14]	validation-rmse:8.00009                         
[15]	validation-rmse:7.87065                         
[16]	validation-rmse:7.75433                         
[17]	validation-rmse:7.64883                         
[18]	validation-rmse:7.54891

### MLFLOW using xgboost ```.autolog```

In [34]:
# Best model using autolog

params = {
    'max_depth': 55,
    'learning_rate': 0.36911726039982323, 
    'reg_alpha': 0.03600652939958025,
    'reg_lambda': 0.003501545963372365,
    'min_child_weight': 1.627889882749392,
    'objective': 'reg:linear',
    'seed': 42
}


mlflow.xgboost.autolog() # This register everything in an automated way!

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, "validation")],
    early_stopping_rounds=5
)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("mae", mae)

2024/06/16 10:55:27 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '39347f9dd7ff4b409a4be778fe6f57df', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:55:59] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.37770
[1]	validation-rmse:7.90854
[2]	validation-rmse:7.18834
[3]	validation-rmse:6.84450
[4]	validation-rmse:6.67290
[5]	validation-rmse:6.58317
[6]	validation-rmse:6.52981
[7]	validation-rmse:6.50077
[8]	validation-rmse:6.47926
[9]	validation-rmse:6.46833
[10]	validation-rmse:6.45939
[11]	validation-rmse:6.45518
[12]	validation-rmse:6.45012
[13]	validation-rmse:6.44694
[14]	validation-rmse:6.44451
[15]	validation-rmse:6.43960
[16]	validation-rmse:6.43461
[17]	validation-rmse:6.43222
[18]	validation-rmse:6.42794
[19]	validation-rmse:6.42685
[20]	validation-rmse:6.42367
[21]	validation-rmse:6.41987
[22]	validation-rmse:6.41792
[23]	validation-rmse:6.41612
[24]	validation-rmse:6.41245
[25]	validation-rmse:6.40846
[26]	validation-rmse:6.40694
[27]	validation-rmse:6.40435
[28]	validation-rmse:6.40110
[29]	validation-rmse:6.40015
[30]	validation-rmse:6.39882
[31]	validation-rmse:6.39520
[32]	validation-rmse:6.39302
[33]	validation-rmse:6.39049
[34]	validation-rmse:6.3

2024/06/16 11:03:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/16 11:03:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:03:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


### MOST USED! --> XGboost - ```.log_model``` and Preprocessor and Make Inference from MLflow Artifact

In [11]:
# Best model using autolog

best_params = {
    'max_depth': 55,
    'learning_rate': 0.36911726039982323, 
    'reg_alpha': 0.03600652939958025,
    'reg_lambda': 0.003501545963372365,
    'min_child_weight': 1.627889882749392,
    'objective': 'reg:linear',
    'seed': 42
}


mlflow.log_params(best_params)

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, "validation")],
    early_stopping_rounds=4
)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("mae", mae)

with open("../models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:28:56] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.37770
[1]	validation-rmse:7.90854
[2]	validation-rmse:7.18834
[3]	validation-rmse:6.84450
[4]	validation-rmse:6.67290
[5]	validation-rmse:6.58317
[6]	validation-rmse:6.52981
[7]	validation-rmse:6.50077
[8]	validation-rmse:6.47926
[9]	validation-rmse:6.46833
[10]	validation-rmse:6.45939
[11]	validation-rmse:6.45518
[12]	validation-rmse:6.45012
[13]	validation-rmse:6.44694
[14]	validation-rmse:6.44451
[15]	validation-rmse:6.43960
[16]	validation-rmse:6.43461
[17]	validation-rmse:6.43222
[18]	validation-rmse:6.42794
[19]	validation-rmse:6.42685
[20]	validation-rmse:6.42367
[21]	validation-rmse:6.41987
[22]	validation-rmse:6.41792
[23]	validation-rmse:6.41612
[24]	validation-rmse:6.41245
[25]	validation-rmse:6.40846
[26]	validation-rmse:6.40694
[27]	validation-rmse:6.40435
[28]	validation-rmse:6.40110
[29]	validation-rmse:6.40015
[30]	validation-rmse:6.39882
[31]	validation-rmse:6.39520
[32]	validation-rmse:6.39302
[33]	validation-rmse:6.39049
[34]	validation-rmse:6.3

/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:31:24] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools 

In [12]:
# It was copied from MLflow "Artifacts" --> "Make Predictions"

logged_model = 'runs:/aedd49e2a0064a7cb4b8a68f6e671964/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:38:16] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [13]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: aedd49e2a0064a7cb4b8a68f6e671964

In [14]:
# Loaded as a xgboost flavor. mlflow supports different ways
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

/home/dpalacioj/miniconda3/envs/study_mlops/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:40:00] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [15]:
#y_pred = booster.predict(valid)

y_pred = xgboost_model.predict(valid)

In [16]:
y_pred[:10]

array([16.76273 , 11.787159,  9.080584, 18.734266, 13.332366, 23.764797,
       30.536175, 10.114101,  8.843522, 15.797203], dtype=float32)